In [1]:
# some code implemented from youtube tutorial from Madhav Thaker
# https://github.com/madhavthaker/spotify-recommendation-system

In [2]:
import numpy as np
import pandas as pd
import json
from IPython.display import Image
import itertools
import sys
import re
import os
import matplotlib.pyplot as plt

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")


url = os.getcwd()

In [3]:
from secrets import client_id, client_secret

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

# Spotify access
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8881/')
sp = spotipy.Spotify(auth=token)

In [10]:
def get_users_playlists():
    playlist_names = {}
    playlist_photos = {}
    
    for i in sp.current_user_playlists()['items']:
        playlist_names[i['name']] = i['uri'].split(':')[2]
        playlist_photos[i['uri'].split(':')[2]] = i['images'][0]['url']
    
    return playlist_names, playlist_photos

# Call get_users_playlists()
playlist_names, playlist_photos = get_users_playlists()

In [6]:
#playlist_names

In [7]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [79]:
def get_track_info(number):
    # Track
    track = tracks[number]
    added_by = track['added_by']['id']
    track_info = track['track']

    # Informations about the track
    track_id = track_info['id']
    track_name = track_info['name']
    track_popularity = track_info['popularity']
    track_link = track_info['external_urls']['spotify']
    track_preview_url = track_info['preview_url']

    # Informations about the artists
    track_artist = track_info['artists'][0]
    track_artist_id = track_artist['id']
    track_artist_name = track_artist['name']


    #print(track_info.keys())
    print(track_name)
    print(track_popularity)
    print(track_link)
    #print(track_info[''])
    #print(track_info)

    #final step
    playlist_tracks[playlist_name] = tracks


In [87]:
playlist_tracks = {}

for playlist_name, playlist_id in playlist_names.items():
    if(str(playlist_id) == "5Yi5I6uqMn9oyV3RhvUVNm"):
        tracks = get_playlist_tracks("davitoz", playlist_id)
        get_track_info(2)


The Passenger
74
https://open.spotify.com/track/15BQ7vEDv2LJuh8TxWIhtd


In [17]:
print({k: playlist_tracks[k] for k in list(playlist_tracks)[:2]})

{'New Playlist 2': [{'added_at': '2021-03-27T21:53:39Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/davitoz'}, 'href': 'https://api.spotify.com/v1/users/davitoz', 'id': 'davitoz', 'type': 'user', 'uri': 'spotify:user:davitoz'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6Jrxnp0JgqmeUX1veU591p'}, 'href': 'https://api.spotify.com/v1/artists/6Jrxnp0JgqmeUX1veU591p', 'id': '6Jrxnp0JgqmeUX1veU591p', 'name': 'Santigold', 'type': 'artist', 'uri': 'spotify:artist:6Jrxnp0JgqmeUX1veU591p'}], 'available_markets': ['AD', 'AE', 'AL', 'AM', 'AO', 'AT', 'AU', 'AZ', 'BA', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BW', 'BY', 'BZ', 'CH', 'CI', 'CM', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DZ', 'EE', 'EG', 'ES', 'FI', 'FR', 'GA', 'GB', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'HN', 'HR', 'HU', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'KE', 'KG', '

In [21]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [23]:
playlist_EDM = create_necessary_outputs('From edits', playlist_names,spotify_df)
playlist_chill = create_necessary_outputs('Eyeex',playlist_names, spotify_df)
#playlist_classical = create_necessary_outputs('Epic Classical',id_name, spotify_df)

NameError: name 'spotify_df' is not defined